In [ ]:
# read pred_filename and filter by scores > 0.5



In [6]:
import json 
import os 
from collections import defaultdict
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval

pred_filename = r'../annotations/confusion_matrix/glip/glip.json'
gt_filename = r'../annotations/confusion_matrix/glip/modgt_glip.json'

temp_pred_filename = r'../annotations/confusion_matrix/temp/temp_preds.json'
temp_gt_filename = r'../annotations/confusion_matrix/temp/temp_gt.json'

preds = json.load(open(pred_filename, 'r'))
gts = json.load(open(gt_filename, 'r'))

determiners = ["a", "an", "the", "any", "all", "no", "every", "each", "my", "your", "our", "this", "these", "that", "those", "few", "several", "many", "some", "little", "much", "both", "neither", "either", "half"]

temp_preds = "temp_preds.json"
temp_gt = "temp_gt.json"

images = gts["images"]

img_det_map = defaultdict(list)
gt_det_map = defaultdict(list)
preds_det_map = defaultdict(list)

for img in images:
    img_id = img["id"]
    det = img["caption"].split()[0]
    img_det_map[img_id] = det

for ann in gts["annotations"]:
    gt_det_map[img_det_map[ann["image_id"]]].append(ann)

for ann in preds:
    preds_det_map[img_det_map[ann["image_id"]]].append(ann)

confusion = defaultdict(lambda: defaultdict(int))
for det in determiners:
    temp_gts = {"images": images, "annotations": gt_det_map[det], "categories": gts["categories"]}
    temp_preds = preds_det_map[det]

    json.dump(temp_gts, open(temp_gt_filename, 'w'))
    json.dump(temp_preds, open(temp_pred_filename, 'w'))

    cocoGt = COCO(temp_gt_filename)
    cocoDt = cocoGt.loadRes(temp_pred_filename)

    annType = "bbox"

    cocoEval = COCOeval(cocoGt, cocoDt, annType)
   
    cocoEval.params.imgIds = sorted(cocoGt.getImgIds())
    cocoEval.evaluate()
    cocoEval.accumulate()
    cocoEval.summarize()
    tp, fp, fn = cocoEval.get_confusion()

    confusion[det]["tp"] = tp
    confusion[det]["fp"] = fp
    confusion[det]["fn"] = fn

for det in determiners: 
    print(det, "TP: ", confusion[det]["tp"], "FP: ", confusion[det]["fp"], "FN: ", confusion[det]["fn"])

loading annotations into memory...
Done (t=0.14s)
creating index...
index created!
Loading and preparing results...
DONE (t=1.31s)
creating index...
index created!
hoooo
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=9.27s).
Accumulating evaluation results...
DONE (t=1.67s).
[0.02745763 0.02745763 0.02745763 ... 0.         0.         0.        ]
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.001
[0.02745763 0.02745763 0.02745763 0.02745763 0.02745763 0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.  

In [4]:
determiners = ["a", "an", "the", "any", "all", "no", "every", "each", "my", "your", "our", "this", "these", "that", "those", "few", "several", "many", "some", "little", "much", "both", "neither", "either", "half"]
print(len(determiners))

25


In [10]:
for det in determiners: 
    tp = confusion[det]["tp"]
    fp = confusion[det]["fp"]
    fn = confusion[det]["fn"]

    if tp > fp and tp > fn: 
        tp = f"\\textcolor{{blue}}{{\\textbf{{{tp}}}}}"
    elif fp > tp and fp > fn:
        fp = f"\\textcolor{{blue}}{{\\textbf{{{fp}}}}}"
    else: 
        fn = f"\\textcolor{{blue}}{{\\textbf{{{fn}}}}}"
    print(det, fn,"&", fp, "&", tp)

a 1948.1 , \textcolor{blue}{\textbf{2965.1}} , 51.9
an 1564.1 , \textcolor{blue}{\textbf{3396.1}} , 435.9
the 1824.6 , \textcolor{blue}{\textbf{3410.6}} , 175.4
any \textcolor{blue}{\textbf{3884.8}} , 2726.8 , 410.2
all \textcolor{blue}{\textbf{6326.1}} , 2521.1 , 598.9
no \textcolor{blue}{\textbf{5453.9}} , 2712.9 , 492.1
every \textcolor{blue}{\textbf{7162.7}} , 2370.7 , 860.3
each \textcolor{blue}{\textbf{6312.4}} , 2491.4 , 757.6
my \textcolor{blue}{\textbf{3731.3}} , 2383.3 , 275.7
your \textcolor{blue}{\textbf{3507.2}} , 2671.2 , 427.8
our \textcolor{blue}{\textbf{7432.4}} , 1704.4 , 540.6
this 1855.3 , \textcolor{blue}{\textbf{2500.3}} , 144.7
these \textcolor{blue}{\textbf{5468.3}} , 1852.3 , 605.7
that 1830.4 , \textcolor{blue}{\textbf{2444.4}} , 169.6
those \textcolor{blue}{\textbf{5380.2}} , 2105.2 , 592.8
few \textcolor{blue}{\textbf{3855.4}} , 3025.4 , 539.6
several \textcolor{blue}{\textbf{10301.4}} , 1344.4 , 1104.6
many \textcolor{blue}{\textbf{15222.9}} , 705.9 , 1538.